In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost



In [2]:
DATA_SOURCE = "../data_assets/csv_files/"
student_df = pd.read_csv(f"{DATA_SOURCE}students_data.csv")
student_historical_df = pd.read_csv(f"{DATA_SOURCE}student_historical_results.csv")
student_info = pd.read_csv(f"{DATA_SOURCE}student_more_info.csv")
jamb_result = pd.read_csv(f"{DATA_SOURCE}jamb_results.csv")
wassce_results = pd.read_csv(f"{DATA_SOURCE}wassce_results.csv")
stud_results = pd.read_csv(f"{DATA_SOURCE}student_results.csv")

student_df['Clubs'] = student_df['Clubs'].str.replace(r'[\{\}]', '', regex=True)

In [3]:
wassce_results.head()

,Student_ID,Year_Of_Admission,WASSCE_Year,Discipline_ID,Discipline,Aspiring_Profession,Gender,English,Mathematics,Civic_Education,...,Commerce,Marketing,Dyeing_and_bleaching,Physics,Chemistry,Food_Nutrition,Biology,Electrical_work,Agriculture,Technical_Drawing
0,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,1,Art,Writer,Female,C,B,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8212a84c-bd64-41a7-9666-39e1fe7e2660,2018,2021,2,Commercial,Entrepreneur,Male,B,A,A,...,C,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5037d342-53b0-487e-969e-145076f49386,2018,2021,2,Commercial,Economist,Male,D,B,C,...,C,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00ceff36-18df-4cc5-bed4-3b144a3ea197,2018,2021,3,Science,Food Nutritionist,Female,E,C,A,...,NaN,NaN,C,C,C,NaN,NaN,NaN,NaN,NaN
4,c6aef2d3-74fb-4b5d-9949-7629511416a7,2018,2021,3,Science,Biologist,Male,F,D,B,...,NaN,D,B,C,NaN,C,NaN,NaN,NaN,NaN


In [4]:
stud_results.head()

,Student_ID,Class_Grade_ID,Session,Subject,Score
0,08423978-1a04-4812-87ed-cd91cd8badb3,2,1,English,84.00
1,08423978-1a04-4812-87ed-cd91cd8badb3,2,1,Maths,90.08
2,08423978-1a04-4812-87ed-cd91cd8badb3,2,1,Civic Education,53.91
3,08423978-1a04-4812-87ed-cd91cd8badb3,2,1,ICT,64.18
4,08423978-1a04-4812-87ed-cd91cd8badb3,2,1,Physics,50.57


In [5]:
student_historical_df.head()

,Student_ID,Grade,Session,Subject,Score,Score_Type
0,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,English,50.01,Current
1,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,Maths,67.25,Current
2,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,Civic Education,79.99,Current
3,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,ICT,88.92,Current
4,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,Literature,52.39,Current


In [58]:
grade_3_students = student_df[student_df['Class_Grade_ID'] == 3]

historical_results = student_historical_df.merge(
    grade_3_students[['Student_ID', 'Discipline_ID', 'Disability', 'Parent_Income_Level', 'Class_Prefect']],
    on='Student_ID',
    how='left'
)

In [59]:
historical_results = historical_results.merge(
    student_info,
    on="Student_ID",
    how="left"
)

historical_results.head()

,Student_ID,Grade,Session,Subject,Score,Score_Type,Discipline_ID,Disability,Parent_Income_Level,Class_Prefect,Attendance_Rate,Study_Hours,Subject_Struggle,Struggle_Scale,Class_Enjoyment,Understanding_Level
0,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,English,50.01,Current,1.0,No,Low,No,0.92,5,Civic Education,5,4,2
1,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,Maths,67.25,Current,1.0,No,Low,No,0.92,5,Civic Education,5,4,2
2,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,Civic Education,79.99,Current,1.0,No,Low,No,0.92,5,Civic Education,5,4,2
3,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,ICT,88.92,Current,1.0,No,Low,No,0.92,5,Civic Education,5,4,2
4,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,Literature,52.39,Current,1.0,No,Low,No,0.92,5,Civic Education,5,4,2


In [12]:
def process_student_scores(df):
    # Assuming 'English' is the subject column and 'Score' contains scores
    # Year_Of_Admission	WASSCE_Year	Discipline_ID	Discipline	Aspiring_Profession	Gender
    records = []
    
    for index, row in df.iterrows():
        student_id = row['Student_ID']
        admission = row['Year_Of_Admission']
        wassce_year = row['WASSCE_Year']
        discipline_id = row['Discipline_ID']
        discipline = row['Discipline']
        aspiring = row['Aspiring_Profession']
        gender = row['Gender']

        # Create an entry for each subject the student has a score for
        for subject, score in row.items():
            if subject not in ['Student_ID', 'Year_Of_Admission', 'WASSCE_Year', 'Discipline_ID', 'Discipline', 'Aspiring_Profession', 'Gender']:
                if pd.notna(score):  # Ensure the score is valid (not NaN)
                    record = {
                        'Student_ID': student_id,
                        'Year_Of_Admission': admission,
                        'WASSCE_Year': wassce_year,
                        'Subject': subject,
                        'Score': score,
                        'Discipline_ID': discipline_id,
                        'Discipline': discipline,
                        'Aspiring_Profession': aspiring,
                        'Gender': gender
                    }
                    records.append(record)
    
    # Convert list of records to a DataFrame
    return pd.DataFrame(records)



processed_df = process_student_scores(wassce_results)
print(processed_df)

                                Student_ID  Year_Of_Admission  WASSCE_Year  \
0     86ff68ca-88c1-433e-8e02-2ee2f28476d5               2018         2021   
1     86ff68ca-88c1-433e-8e02-2ee2f28476d5               2018         2021   
2     86ff68ca-88c1-433e-8e02-2ee2f28476d5               2018         2021   
3     86ff68ca-88c1-433e-8e02-2ee2f28476d5               2018         2021   
4     86ff68ca-88c1-433e-8e02-2ee2f28476d5               2018         2021   
...                                    ...                ...          ...   
3898  a591ef24-d758-4467-a517-98b0e3d40975               2020         2023   
3899  a591ef24-d758-4467-a517-98b0e3d40975               2020         2023   
3900  a591ef24-d758-4467-a517-98b0e3d40975               2020         2023   
3901  a591ef24-d758-4467-a517-98b0e3d40975               2020         2023   
3902  a591ef24-d758-4467-a517-98b0e3d40975               2020         2023   

                   Subject Score  Discipline_ID Discipline  \
0

In [13]:
processed_df

,Student_ID,Year_Of_Admission,WASSCE_Year,Subject,Score,Discipline_ID,Discipline,Aspiring_Profession,Gender
0,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,English,C,1,Art,Writer,Female
1,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,Mathematics,B,1,Art,Writer,Female
2,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,Civic_Education,C,1,Art,Writer,Female
3,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,ICT,B,1,Art,Writer,Female
4,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,Literature,C,1,Art,Writer,Female
...,...,...,...,...,...,...,...,...,...
3898,a591ef24-d758-4467-a517-98b0e3d40975,2020,2023,ICT,B,3,Science,Biologist,Male
3899,a591ef24-d758-4467-a517-98b0e3d40975,2020,2023,_French_Igbo,A,3,Science,Biologist,Male
3900,a591ef24-d758-4467-a517-98b0e3d40975,2020,2023,Dyeing_and_bleaching,C,3,Science,Biologist,Male
3901,a591ef24-d758-4467-a517-98b0e3d40975,2020,2023,Physics,E,3,Science,Biologist,Male


In [14]:
def map_scores(df):
    score_mapping = {
        'A': 80,
        'B': (60, 79),
        'C': (50, 59),
        'D': (40, 49),
        'E': (30, 39),
        'F': (0, 29)
    }

    def grade_to_numeric(grade):
        if grade == 'A':
            return score_mapping['A']
        elif grade == 'B':
            return (score_mapping['B'][0] + score_mapping['B'][1]) / 2
        elif grade == 'C':
            return (score_mapping['C'][0] + score_mapping['C'][1]) / 2
        elif grade == 'D':
            return (score_mapping['D'][0] + score_mapping['D'][1]) / 2
        elif grade == 'E':
            return (score_mapping['E'][0] + score_mapping['E'][1]) / 2
        elif grade == 'F':
            return (score_mapping['F'][0] + score_mapping['F'][1]) / 2
        else:
            return None

    df['Numeric_Score'] = df['Score'].apply(grade_to_numeric)
    
    return df


df = pd.DataFrame(processed_df)

df_mapped = map_scores(df)

df_mapped.head(2)

,Student_ID,Year_Of_Admission,WASSCE_Year,Subject,Score,Discipline_ID,Discipline,Aspiring_Profession,Gender,Numeric_Score
0,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,English,C,1,Art,Writer,Female,54.5
1,86ff68ca-88c1-433e-8e02-2ee2f28476d5,2018,2021,Mathematics,B,1,Art,Writer,Female,69.5


In [16]:
df_mapped.to_csv("wassce_students_new.csv", index=False)

In [17]:
historical_results.to_csv("Current_students_results.csv", index=False)

In [35]:
historical_results.head(1)

,Student_ID,Grade,Session,Subject,Score,Score_Type,Discipline_ID,Disability,Parent_Income_Level,Class_Prefect,Attendance_Rate,Study_Hours,Subject_Struggle,Struggle_Scale,Class_Enjoyment,Understanding_Level,Predicted_Final_Score,Passed_Mandatory,Passed_Five_Subjects
0,003fe7d6-2425-41d9-8983-e55c1d78a34d,3,1,9,50.01,Current,1.0,0,0,0,0.92,5,3,5,4,2,70.903694,False,True


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [83]:
income_mapping = {
    'Low':1,
    'Middle Class': 2,
    'Upper Class': 3

}

disability_mapping = {
    'No': 0,
    'Yes': 1
}

historical_results['Parent_Income_Level_Num'] = historical_results['Parent_Income_Level'].map(income_mapping)
historical_results["Disability_Num"] = historical_results["Disability"].map(disability_mapping)

In [84]:
grade_3_students = historical_results[historical_results['Grade'] == 3]
class_mapping = {
    'No': 0,
    'Yes': 1
}

grade_3_students["Class_Prefect_Num"] = grade_3_students["Class_Prefect"].map(class_mapping)
grade_3_students_ids = grade_3_students['Student_ID']
grade_3_subject_ids = grade_3_students['Subject']
grade_3_discipline_ids = grade_3_students['Discipline_ID']
grade_3_grade_ids = grade_3_students['Grade']                 


C:\Users\A\AppData\Local\Temp\ipykernel_41708\4123804784.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grade_3_students["Class_Prefect_Num"] = grade_3_students["Class_Prefect"].map(class_mapping)


In [85]:
# # Encode categorical variables
# label_encoders = {}
# for column in ['Parent_Income_Level', 'Class_Prefect', 'Disability',]:
#     le = LabelEncoder()
#     grade_3_students[column] = le.fit_transform(grade_3_students[column])
#     label_encoders[column] = le

target = 'Score'
# Feature and target variables
X = grade_3_students.drop(columns=['Student_ID', 'Subject_Struggle', 'Score', 'Score_Type', 'Session', 'Subject', 'Grade', 'Discipline_ID', 'Parent_Income_Level', 'Disability', 'Class_Prefect'])  # Drop ID and target columns
y = grade_3_students[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 307.4781420393238


In [86]:
X.columns

Index(['Attendance_Rate', 'Study_Hours', 'Struggle_Scale', 'Class_Enjoyment',
       'Understanding_Level', 'Parent_Income_Level_Num', 'Disability_Num',
       'Class_Prefect_Num'],
      dtype='object')

In [89]:
# Predict the final scores for all subjects for each student
train_features = ['Attendance_Rate', 'Study_Hours', 'Struggle_Scale', 'Class_Enjoyment',
       'Understanding_Level', 'Parent_Income_Level_Num', 'Disability_Num',
       'Class_Prefect_Num']


grade_3_students['Predicted_Final_Score'] = xgb_model.predict(grade_3_students[train_features])
grade_3_students['Student_ID'] = grade_3_students_ids
grade_3_students['Subject'] =  grade_3_subject_ids
grade_3_students["Grade"] = grade_3_grade_ids
grade_3_students["Discipline_ID"] = grade_3_discipline_ids

# Define mandatory subjects
mandatory_subjects = ['English', 'Math', 'Civic Education', 'ICT']

# Define a function to check if a student has passed the mandatory subjects
def passed_mandatory_subjects(df):
    passed_subjects = df[(df['Subject'].isin(mandatory_subjects)) & (df['Predicted_Final_Score'] >= 50)]
    return len(passed_subjects) == len(mandatory_subjects)

# Apply the condition for passing mandatory subjects for each student and map the result back to the DataFrame
grade_3_students['Passed_Mandatory'] = grade_3_students.groupby('Student_ID').apply(
    lambda group: passed_mandatory_subjects(group)
).reset_index(drop=True)

# Now, check if the student has passed 5 or more subjects (scored ≥50)
def passed_five_subjects(df):
    passed_subjects = df[df['Predicted_Final_Score'] >= 50]
    return len(passed_subjects) >= 5

# Apply the check for students who passed 5 or more subjects and map the result back to the DataFrame
grade_3_students['Passed_Five_Subjects'] = grade_3_students.groupby('Student_ID').apply(
    lambda group: passed_five_subjects(group)
).reset_index(drop=True)

# Filter students who are at risk by passing less than 5 subjects or failing any mandatory subject
at_risk_students = grade_3_students[
    (grade_3_students['Passed_Mandatory'] != True) | (grade_3_students['Passed_Five_Subjects'] != True)
]


C:\Users\A\AppData\Local\Temp\ipykernel_41708\388363984.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grade_3_students['Predicted_Final_Score'] = xgb_model.predict(grade_3_students[train_features])
C:\Users\A\AppData\Local\Temp\ipykernel_41708\388363984.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grade_3_students['Student_ID'] = grade_3_students_ids
C:\Users\A\AppData\Local\Temp\ipykernel_41708\388363984.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [90]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()

pdf.set_font('Arial', 'B', 12)
pdf.cell(200, 10, txt="At-Risk Students Report", ln=True, align='C')

pdf.set_font('Arial', '', 12)
# Suggest interventions for at-risk students
for index, student in at_risk_students.iterrows():
    print(f"Student ID: {student['Student_ID']} - Suggested Interventions: Personalized tutoring, "
          f"special attention to students with disabilities, and increased study hours.")
pdf.output("wassce_at_risk_students_report.pdf")
print("PDF report created successfully!")

Student ID: 003fe7d6-2425-41d9-8983-e55c1d78a34d - Suggested Interventions: Personalized tutoring, special attention to students with disabilities, and increased study hours.
Student ID: 003fe7d6-2425-41d9-8983-e55c1d78a34d - Suggested Interventions: Personalized tutoring, special attention to students with disabilities, and increased study hours.
Student ID: 003fe7d6-2425-41d9-8983-e55c1d78a34d - Suggested Interventions: Personalized tutoring, special attention to students with disabilities, and increased study hours.
Student ID: 003fe7d6-2425-41d9-8983-e55c1d78a34d - Suggested Interventions: Personalized tutoring, special attention to students with disabilities, and increased study hours.
Student ID: 003fe7d6-2425-41d9-8983-e55c1d78a34d - Suggested Interventions: Personalized tutoring, special attention to students with disabilities, and increased study hours.
Student ID: 003fe7d6-2425-41d9-8983-e55c1d78a34d - Suggested Interventions: Personalized tutoring, special attention to stude

In [91]:
from sklearn.metrics import r2_score



# # Reattach Student_ID and Subject for final display
# student_ids = historical_results['Student_ID']
# subjects = historical_results['Subject']

# Predict final scores
grade_3_students['Predicted_Final_Score'] = xgb_model.predict(grade_3_students[train_features])

# Calculate R² score (true scores vs predicted scores)
r2 = r2_score(grade_3_students['Score'], grade_3_students['Predicted_Final_Score'])
print(f"R² Score: {r2}")

# Create a DataFrame to display Student_ID, Subject, and Predicted_Final_Score
result_df = grade_3_students[['Student_ID', 'Subject', 'Predicted_Final_Score']]



R² Score: -0.002300129390451211


C:\Users\A\AppData\Local\Temp\ipykernel_41708\979053858.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grade_3_students['Predicted_Final_Score'] = xgb_model.predict(grade_3_students[train_features])


In [ ]:
from sklearn.model_selection import GridSearchCV

# Example hyperparameters to tune
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [100, 200, 300]
}

# Initialize XGBoost model
xgb_model = XGBRegressor()

# Grid search to find best hyperparameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='r2', cv=5, verbose=1)
grid_search.fit(X_train, y_train)

# Best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² Score after tuning: {r2}")


In [ ]:
# Display the result DataFrame
result_df.head()

In [54]:
historical_results.shape

(76026, 19)

In [56]:
from sklearn.metrics import r2_score

# Filter for Grade 3 students
grade_3_students = historical_results[historical_results['Grade'] == 3]

# Predict the final scores for all subjects for Grade 3 students
train_features = ['Grade', 'Subject', 'Discipline_ID', 'Disability', 'Parent_Income_Level', 
                  'Class_Prefect', 'Attendance_Rate', 'Study_Hours', 'Subject_Struggle',
                  'Struggle_Scale', 'Class_Enjoyment', 'Understanding_Level']

# Predict final scores for Grade 3 students
grade_3_students['Predicted_Final_Score'] = xgb_model.predict(grade_3_students[train_features])

# Calculate R² score for Grade 3 students (true scores vs predicted scores)
r2 = r2_score(grade_3_students['Score'], grade_3_students['Predicted_Final_Score'])
print(f"R² Score for Grade 3 students: {r2}")

# Create a DataFrame to display Student_ID, Subject, and Predicted_Final_Score for Grade 3 students
grade_3_result_df = grade_3_students[['Student_ID', 'Subject', 'Predicted_Final_Score']]



# If you want to return it as a DataFrame in a Jupyter Notebook or other environment
# grade_3_result_df.head()  # To display the first few rows


R² Score for Grade 3 students: -0.0038858309700919857


C:\Users\A\AppData\Local\Temp\ipykernel_41708\2257535253.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grade_3_students['Predicted_Final_Score'] = xgb_model.predict(grade_3_students[train_features])


In [57]:
# Display the result DataFrame for Grade 3 students
grade_3_result_df.head()

,Student_ID,Subject,Predicted_Final_Score
0,003fe7d6-2425-41d9-8983-e55c1d78a34d,9,70.849762
1,003fe7d6-2425-41d9-8983-e55c1d78a34d,16,70.849762
2,003fe7d6-2425-41d9-8983-e55c1d78a34d,4,70.849762
3,003fe7d6-2425-41d9-8983-e55c1d78a34d,12,70.849762
4,003fe7d6-2425-41d9-8983-e55c1d78a34d,14,70.849762
